<a href="https://colab.research.google.com/github/Minteb/Hyperspectral-Image-Classification-using-Random-Forest-Algorithm/blob/main/Landsat8_NDVI_2013_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap
import os
from google.colab import files

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='spiaproject')

In [2]:
# Define ROI (adjust coordinates as needed)
roi = ee.Geometry.Polygon(
    [[[38.5, 8.8],
      [38.5, 9.2],
      [39.0, 9.2],
      [39.0, 8.8]]])

In [3]:
# Cloud masking function
def mask_clouds(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
           .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

In [4]:
# NDVI calculation
def add_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return image.addBands(ndvi)

In [5]:
# Landsat 8 collection with cloud masking
collection = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
              .filterBounds(roi)
              .filterDate('2014-01-01', '2024-12-31')
              .map(mask_clouds)
              .map(add_ndvi)
              .select('NDVI'))

In [6]:
# Create monthly composites
def create_monthly_composites(year, month):
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')

    monthly = collection.filterDate(start_date, end_date)
    composite = monthly.median().clip(roi)

    # Get mean NDVI value (returns a server-side object)
    mean_val = composite.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=30
    ).get('NDVI')

    return composite, mean_val

In [7]:
# Output folder
output_folder = "/content/Landsat_NDVI/"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
for year in range(2014, 2025):
    print(f"\nProcessing year: {year}")
    for month in range(1, 13):
        print(f"Month: {month:02d}", end=' ')

        try:
            composite, mean_val = create_monthly_composites(year, month)

            # Get the actual mean value (client-side)
            mean_val_info = mean_val.getInfo()  # This fetches the value from server

            if mean_val_info is None:
                print("⚠️ No valid data")
                continue

            filename = f"NDVI_{year}_{month:02d}.tif"
            filepath = os.path.join(output_folder, filename)

            # Export
            geemap.ee_export_image(
                composite,
                filename=filepath,
                scale=30,
                region=roi,
                file_per_band=False
            )

            print(f"✅ Exported (Mean NDVI: {mean_val_info:.3f})")

            # Download
            files.download(filepath)

        except Exception as e:
            print(f"❌ Error: {str(e)}")
            continue
print("Processing complete!")

In [ ]:
print("Processing complete!")